<a href="https://colab.research.google.com/github/Sumitkevlani/LLM/blob/main/Stock_Market_Helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/adidror005/youtube-videos/main/small_news_dataset%20(1).csv')
df.head()

,author,content,created_at,headline,id,images,source,summary,symbols,updated_at,url
0,Anan Ashraf,<p><strong>Future Fund&nbsp;</strong>Managing ...,2024-05-18T08:03:56Z,Tesla Bull Gary Black Says 'Likely' Approval O...,38907041,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Gary Black predicts shareholders will likely a...,['TSLA'],2024-05-18T08:04:55Z,https://www.benzinga.com/news/24/05/38907041/t...
1,Anusuya Lahiri,"<p>This past week, eight companies in&nbsp;<st...",2024-05-06T18:59:43Z,"Thanks To Apple, This Analyst Is Optimistic On...",38651469,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Key Insights from KeyBanc: Semiconductor Secto...,"['AAPL', 'AMD', 'ARM', 'ARW', 'AVT', 'CRUS', '...",2024-05-06T18:59:43Z,https://www.benzinga.com/analyst-ratings/analy...
2,Benzinga Neuro,"<p><strong>Ross Gerber</strong>, the CEO of <s...",2024-04-18T05:21:01Z,Tesla Board's Credibility At Stake With $56B E...,38306906,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,"Gerber, in an interview stated, &#34;I&#39;m s...",['TSLA'],2024-04-18T05:21:02Z,https://www.benzinga.com/markets/equities/24/0...
3,Vaishali Prayag,"<p><strong>Recursion Pharmaceuticals, Inc.</st...",2024-05-13T20:51:35Z,Recursion Pharmaceuticals Stock Surges 10% As ...,38802247,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,"Recursion Pharmaceuticals, Inc. (NASDAQ: RXRX)...","['NVDA', 'RXRX']",2024-05-13T20:51:36Z,https://www.benzinga.com/news/24/05/38802247/r...
4,Anusuya Lahiri,<p><strong>Nvidia Corp</strong> (NASDAQ:<a cla...,2024-04-22T12:44:44Z,Nvidia Joins Forces with Japan's Tech Institut...,38355877,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Nvidia partners with Japan&#39;s National Inst...,"['AMZN', 'MSFT', 'NVDA', 'TSM']",2024-04-22T12:44:44Z,https://www.benzinga.com/government/24/04/3835...


In [ ]:
!pip install transformers einops accelerate langchain bitsandbytes

In [ ]:
!pip install langchain-huggingface

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install llama-index

In [ ]:
!pip install llama-index-llms-huggingface

In [ ]:
!pip install llama-index-embeddings-langchain

In [7]:
from llama_index.core import Document, VectorStoreIndex

documents = []
for index,document in df.iterrows():
  content = document['content']
  row_dict = document.to_dict()
  meta_data_dict = {k:v for k,v in row_dict.items() if k in ['created_at','author']}
  doc = Document(text=content,metadata=meta_data_dict)
  documents.append(doc)

In [8]:
from llama_index.core import ServiceContext

In [9]:
from llama_index.llms.huggingface import HuggingFaceLLM

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [10]:
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [11]:
system_prompt = "You are a helpful AI assistant. Your goal is to answer the questions based on the instructions and the context provided."
query_wrapper_format = "<|USER|>{query_str}<|ASSISTANT|>"

In [ ]:
!!huggingface-cli login

In [14]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_format,
    generate_kwargs={"temperature": 0.25, "do_sample": False},
    tokenizer_name="mistralai/Mistral-7B-v0.1",
    model_name="mistralai/Mistral-7B-v0.1",
    device_map="auto",
    model_kwargs={"torch_dtype":torch.float16,"load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding
embd_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [21]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embd_model,
)

<ipython-input-21-4d4960ab54ba>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [22]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=2048, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7d9c21edbd30>, num_workers=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7d9c21edbd30>, id_func=<function default_id_func at 0x7d9d73108ca0>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7d9d750cf880>, callback_manager=<llama_index

In [24]:
index = VectorStoreIndex.from_documents(documents,service_context=service_context)

In [25]:
query_engine = index.as_query_engine()

In [27]:
query_engine.query("Who is the CEO of Gerber Kawasaki Wealth and Investment Management")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response(response='Ross Gerber is the CEO of Gerber Kawasaki Wealth and Investment Management.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', source_nodes=[NodeWithScore(node=TextNode(id_='fcc9dbcd-57ae-4b63-a4e2-1a2d6635197e', embedding=None, metadata={'author': 'Benzinga Neuro', 'created_at': '2024-04-18T05:21:01Z'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9128bf1f-f0bc-4f45-9d79-0662a875c4fb', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'author': 'Benzinga Neuro', '